## Download Data

Our work used the open source dataset introduced in  ['Data driven prediciton of battery cycle life before capacity degradation' by K.A. Severson, P.M. Attia, et al](https://www.nature.com/articles/s41560-019-0356-8). This notebook is a modified data processing tool based on the [original code](https://github.com/rdbraatz/data-driven-prediction-of-battery-cycle-life-before-capacity-degradation). 

In [ ]:
import requests
import os

urls = ["https://data.matr.io/1/api/v1/file/5c86c0b5fa2ede00015ddf66/download",
        "https://data.matr.io/1/api/v1/file/5c86bf13fa2ede00015ddd82/download",
        "https://data.matr.io/1/api/v1/file/5c86bd64fa2ede00015ddbb2/download"]

output_paths = ["data/2017-05-12_batchdata_updated_struct_errorcorrect.mat",
               "data/2017-06-30_batchdata_updated_struct_errorcorrect.mat",
               "data/2018-04-12_batchdata_updated_struct_errorcorrect.mat"] 


if not os.path.exists("./data/"):
    os.makedirs("./data/")

    
for i in range(len(urls)):
    
    url = urls[i]
    path = output_paths[i]
    if not os.path.isfile(path):
        response = requests.get(url, stream=True)
        
        if response.status_code == 200:
            with open(path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:  # filter out keep-alive chunks
                        f.write(chunk)
            print(f"Download completed: {path}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
    else:
        print(f"{path.replace('data/','')} is already downloaded in the data folder")


## Convert to PKL

This code converts the raw matplotlib data to pkl files with the features that will be used later for training.

In [ ]:
def process_batch(f, batch, batch_num):
    num_cells = batch['summary'].shape[0]
    bat_dict = {}
    for i in range(num_cells):
        cl = f[batch['cycle_life'][i,0]][()]
        policy = f[batch['policy_readable'][i,0]][()].tobytes()[::2].decode()
        summary_IR = np.hstack(f[batch['summary'][i,0]]['IR'][0,:].tolist())
        summary_QC = np.hstack(f[batch['summary'][i,0]]['QCharge'][0,:].tolist())
        summary_QD = np.hstack(f[batch['summary'][i,0]]['QDischarge'][0,:].tolist())
        summary_TA = np.hstack(f[batch['summary'][i,0]]['Tavg'][0,:].tolist())
        summary_TM = np.hstack(f[batch['summary'][i,0]]['Tmin'][0,:].tolist())
        summary_TX = np.hstack(f[batch['summary'][i,0]]['Tmax'][0,:].tolist())
        summary_CT = np.hstack(f[batch['summary'][i,0]]['chargetime'][0,:].tolist())
        summary_CY = np.hstack(f[batch['summary'][i,0]]['cycle'][0,:].tolist())
        summary = {'IR': summary_IR, 'QC': summary_QC, 'QD': summary_QD, 'Tavg':
                    summary_TA, 'Tmin': summary_TM, 'Tmax': summary_TX, 'chargetime': summary_CT,
                    'cycle': summary_CY}
        cycles = f[batch['cycles'][i,0]]
        cycle_dict = {}
        for j in range(cycles['I'].shape[0]):
            I = np.hstack((f[cycles['I'][j,0]][()]))
            Qc = np.hstack((f[cycles['Qc'][j,0]][()]))
            Qd = np.hstack((f[cycles['Qd'][j,0]][()]))
            Qdlin = np.hstack((f[cycles['Qdlin'][j,0]][()]))
            T = np.hstack((f[cycles['T'][j,0]][()]))
            Tdlin = np.hstack((f[cycles['Tdlin'][j,0]][()]))
            V = np.hstack((f[cycles['V'][j,0]][()]))
            dQdV = np.hstack((f[cycles['discharge_dQdV'][j,0]][()]))
            t = np.hstack((f[cycles['t'][j,0]][()]))
            cd = {'I': I, 'Qc': Qc, 'Qd': Qd, 'Qdlin': Qdlin, 'T': T, 'Tdlin': Tdlin, 'V':V, 'dQdV': dQdV, 't':t}
            cycle_dict[str(j)] = cd

        cell_dict = {'cycle_life': cl, 'charge_policy':policy, 'summary': summary, 'cycles': cycle_dict}
        key = f'b{batch_num}c' + str(i)
        bat_dict[key]=   cell_dict
    return bat_dict

In [ ]:
import h5py
import scipy.io
import numpy as np
import pickle
import os
import csv

local_DIR = './data/'

matFilename_batch1 = os.path.join(local_DIR, '2017-05-12_batchdata_updated_struct_errorcorrect.mat')
matFilename_batch2 = os.path.join(local_DIR, '2017-06-30_batchdata_updated_struct_errorcorrect.mat')
matFilename_batch3 = os.path.join(local_DIR, '2018-04-12_batchdata_updated_struct_errorcorrect.mat')

f_batch1 = h5py.File(matFilename_batch1)
f_batch2 = h5py.File(matFilename_batch2)
f_batch3 = h5py.File(matFilename_batch3)

batch1 = f_batch1['batch']
batch2 = f_batch2['batch']
batch3 = f_batch3['batch']

bat_dict1 = process_batch(f_batch1, batch1, 1)
print("Batch 1/3 complete", end="\r")
bat_dict2 = process_batch(f_batch2, batch2, 2)
print("Batch 2/3 complete", end="\r")
bat_dict3 = process_batch(f_batch3, batch3, 3)
print("Batch 3/3 complete", end="\r")

In [ ]:
# Write the converted files to disk for later use
ROOT_DIR = 'data/'

if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)
    
with open(os.path.join(ROOT_DIR, 'batch1.pkl'),'wb') as fp:
        pickle.dump(bat_dict1,fp)

with open(os.path.join(ROOT_DIR, 'batch2.pkl'),'wb') as fp:
        pickle.dump(bat_dict2,fp)

with open(os.path.join(ROOT_DIR, 'batch3.pkl'),'wb') as fp:
        pickle.dump(bat_dict3,fp)

## Data Cleaning

We replicate the data cleaning from the [original notebook below](https://github.com/rdbraatz/data-driven-prediction-of-battery-cycle-life-before-capacity-degradation/blob/master/Load%20Data.ipynb)

In [ ]:
import pickle
import numpy as np
import pandas as pd
import csv 

ROOT_DIR = 'data/'

batch1 = pickle.load(open(os.path.join(ROOT_DIR, 'batch1.pkl'), 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

batch2 = pickle.load(open(os.path.join(ROOT_DIR, 'batch2.pkl'),'rb'))

# There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
# and put it with the correct cell from batch1
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
add_len = [662, 981, 1060, 208, 482];
for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]


del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']


batch3 = pickle.load(open(os.path.join(ROOT_DIR, 'batch3.pkl'),'rb'))
# remove noisy channels from batch3
del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c42']
del batch3['b3c43']


bat_dict = {**batch1, **batch2, **batch3}

# write metadata out to disk
metadata = [(bat, bat_dict[bat]['cycle_life'][0][0], bat_dict[bat]['charge_policy']) for bat in bat_dict.keys()]
filename = "data/metadata.csv"
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(metadata)

## Create Battery Pkl files

The below code concatenates the charging and discharging segement of each battery's cycle to create a continous capacity curve. It then uses a basic derivative to classify each point as either charging/discharging and adds a new feature dV/dT. The charging and discharging segements of each battery are written to seperate directories.

In [ ]:
import pandas as pd
import os

C_ROOT_DIR = "data/charging/" 
D_ROOT_DIR = "data/discharging/"

if not os.path.exists(C_ROOT_DIR):
    os.makedirs(C_ROOT_DIR)

if not os.path.exists(D_ROOT_DIR):
    os.makedirs(D_ROOT_DIR)


C_bat_dfs = {}
D_bat_dfs = {}

count = 0

for bat in list(bat_dict.keys()):
    
    
    data_for_df = []
    for cycle_num in range(len(bat_dict[bat]['cycles'])):
        idx_switch = -1
        for idx in np.where(bat_dict[bat]['cycles'][str(cycle_num)]['I'] < 0)[0]:
            if (bat_dict[bat]['cycles'][str(cycle_num)]['I'][idx+1] < 0) and (bat_dict[bat]['cycles'][str(cycle_num)]['I'][idx+2] < 0) and (bat_dict[bat]['cycles'][str(cycle_num)]['I'][idx+10] < 0):
                idx_switch = idx
                break
        capacity = np.concatenate((bat_dict[bat]['cycles'][str(cycle_num)]['Qc'][:idx_switch], bat_dict[bat]['cycles'][str(cycle_num)]['Qc'][idx_switch - 1] - bat_dict[bat]['cycles'][str(cycle_num)]['Qd'][idx_switch:]))
        
        # each point that is recorded in a given cycle
        for point_idx in range(len(bat_dict[bat]['cycles'][str(cycle_num)]['I'])):
            data_for_df.append([
                bat_dict[bat]['cycles'][str(cycle_num)]['I'][point_idx],
                bat_dict[bat]['cycles'][str(cycle_num)]['V'][point_idx],
                bat_dict[bat]['cycles'][str(cycle_num)]['T'][point_idx],
                bat_dict[bat]['cycles'][str(cycle_num)]['t'][point_idx],
                capacity[point_idx],
                cycle_num,
                bat_dict[bat]['summary']["IR"][cycle_num]  # add IR
            ])
    
    
    df = pd.DataFrame(data=data_for_df, columns=['I', 'V', 'T', 't', 'Q', 'c', 'IR'])

    # add dVoltage/dTime
    volt = df['V'].to_numpy()
    charge = df['Q'].to_numpy()
    time = df['t'].to_numpy()

    V_ts = pd.Series(volt, index=time)
    charge_ts = pd.Series(charge,index=time)
    
    dV_dt = V_ts.diff(periods=10)
    dV_dt.fillna(0, inplace=True)
    dV_dt = dV_dt.values
    
    dQ_dt = charge_ts.diff(periods=1)
    dQ_dt.fillna(1, inplace=True)
    dQ_dt = dQ_dt.values

    charge_discharge = []
    prev_label = -1
    for val in dQ_dt:
        
        
        if val > 0:
            charge_discharge.append(1)
            prev_label = 1
        elif val < 0:
            charge_discharge.append(0)
            prev_label = 0
        elif val == 0:
            charge_discharge.append(prev_label)
       
    df['charging'] = charge_discharge
    df['dV/dt'] = dV_dt


    charge = []
    discharge = []
    for index, row in df.iterrows():
    
        if row['charging'] == 1:
            charge.append(row.to_list())
        elif row['charging'] == 0:
            discharge.append(row.to_list())

    charge_df = pd.DataFrame(charge, columns=df.columns)
    discharge_df = pd.DataFrame(discharge, columns=df.columns)
    
    charge_df.to_pickle(os.path.join(C_ROOT_DIR, f'{bat}.pkl'))
    discharge_df.to_pickle(os.path.join(D_ROOT_DIR, f'{bat}.pkl'))

    C_bat_dfs[bat] = charge_df
    D_bat_dfs[bat] = discharge_df
    
    count += 1
    print(f'Processed {bat}; {count}/{len(bat_dict.keys())} batteries', end="\r")

Below gives a visual intution to how the battery is being split using the derivative.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# ['b1c32', 'b2c33', 'b2c47']
# Grab data for battery "b1c1"
charge_df = C_bat_dfs['b1c1']
discharge_df = D_bat_dfs['b1c1']

# Filter by target cycle
charge_curve = charge_df[charge_df['c'] == 637]['Q'].reset_index(drop=True)
discharge_curve = discharge_df[discharge_df['c'] == 637]['Q'].reset_index(drop=True)


# Create x-values
charge_x = range(len(charge_curve))
discharge_x = range(len(charge_curve), len(charge_curve) + len(discharge_curve))

# Plot with different colors
plt.plot(charge_x, charge_curve, label='Charge (positive derivative)', color='blue')
plt.plot(discharge_x, discharge_curve, label='Discharge (negative derivative)', color='red')

transition_index = len(charge_curve)
plt.axvline(x=transition_index, color='black', linestyle='--', label='Transition Point')

plt.xticks([])
plt.ylabel('Q', labelpad=5)
plt.legend()

plt.title('Concatenated Charge/Discharge Curve for b1c1 cycle 400')
plt.show()


## Create Train, Validation, and Test Set

The below code creates the train, validation, and test set used in our paper. Other train/validation/test set splits are possible.

In [ ]:
train_bats = []
val_bats = []
test_bats = []

for bat, cycle_life, charging_policy in metadata:
    
    if bat in ['b1c0', 'b1c1', 'b1c2', 'b1c36', 'b1c37', 'b2c17']:
        train_bats.append(bat)
        
    elif bat in ['b1c5', 'b1c33', 'b2c34']:
        val_bats.append(bat)
    elif bat in ['b1c32', 'b2c33', 'b2c47']:
        test_bats.append(bat)
      
print("Train: ", train_bats)
print("Validation: ", val_bats)
print("Test: ", test_bats)

## Write Train, Validation, and Test to Disk

Creates a combinded df of the selected batteries; it does so separately for the charging and discharging batteries.

In [ ]:
import os
import pandas as pd
def save_df(bats, name, ROOT_DIR):
    dfs = []
    for bat in bats:
        df = pd.read_pickle(os.path.join(ROOT_DIR, f'{bat}.pkl'))
        dfs.append(df)

    entire_df = pd.concat(dfs).reset_index(drop=True)
    entire_df.to_pickle(os.path.join(ROOT_DIR, f'{name}.pkl'))

In [ ]:
C_ROOT_DIR = "data/charging/"
D_ROOT_DIR = "data/discharging/"

# charging
save_df(train_bats, "train_df", C_ROOT_DIR)
save_df(val_bats, "val_df", C_ROOT_DIR)
save_df(test_bats, "test_df", C_ROOT_DIR)

# discharging
save_df(train_bats, "train_df", D_ROOT_DIR)
save_df(val_bats, "val_df", D_ROOT_DIR)
save_df(test_bats, "test_df", D_ROOT_DIR)

In [31]:
log = """
Psuedo-label SoC% error per cycle
*****************************************************************
Psuedo-label SoC% error per cycle
*****************************************************************
1 -- FF: 21.16 LSTM: 12.91 Hybrid: 15.61 CL-WR: 15.46
2 -- FF: 19.22 LSTM: 12.82 Hybrid: 14.73 CL-WR: 12.63
3 -- FF: 18.33 LSTM: 12.48 Hybrid: 14.05 CL-WR: 9.54
4 -- FF: 18.78 LSTM: 12.46 Hybrid: 14.3 CL-WR: 8.63
5 -- FF: 19.62 LSTM: 12.53 Hybrid: 14.66 CL-WR: 8.41
6 -- FF: 18.71 LSTM: 13.09 Hybrid: 14.72 CL-WR: 8.21
7 -- FF: 18.56 LSTM: 13.07 Hybrid: 14.71 CL-WR: 7.76
8 -- FF: 18.65 LSTM: 13.2 Hybrid: 14.83 CL-WR: 8.01
9 -- FF: 19.1 LSTM: 13.7 Hybrid: 15.35 CL-WR: 7.7
10 -- FF: 18.56 LSTM: 13.28 Hybrid: 14.89 CL-WR: 7.46
11 -- FF: 17.79 LSTM: 13.24 Hybrid: 14.6 CL-WR: 7.57
12 -- FF: 18.68 LSTM: 13.05 Hybrid: 14.7 CL-WR: 7.63
13 -- FF: 18.91 LSTM: 12.5 Hybrid: 14.34 CL-WR: 8.51
14 -- FF: 19.73 LSTM: 12.69 Hybrid: 14.78 CL-WR: 8.12
15 -- FF: 19.63 LSTM: 12.69 Hybrid: 14.73 CL-WR: 8.11
16 -- FF: 19.34 LSTM: 12.52 Hybrid: 14.53 CL-WR: 8.28
17 -- FF: 18.64 LSTM: 12.82 Hybrid: 14.5 CL-WR: 8.04
18 -- FF: 17.78 LSTM: 13.0 Hybrid: 14.37 CL-WR: 7.34
19 -- FF: 19.37 LSTM: 12.91 Hybrid: 14.87 CL-WR: 7.64
20 -- FF: 18.3 LSTM: 13.38 Hybrid: 14.77 CL-WR: 7.45
21 -- FF: 17.8 LSTM: 13.25 Hybrid: 14.54 CL-WR: 7.3
22 -- FF: 16.09 LSTM: 13.12 Hybrid: 13.93 CL-WR: 7.39
23 -- FF: 15.96 LSTM: 13.12 Hybrid: 13.9 CL-WR: 7.33
24 -- FF: 17.58 LSTM: 13.58 Hybrid: 14.74 CL-WR: 6.93
25 -- FF: 18.2 LSTM: 14.05 Hybrid: 15.29 CL-WR: 6.58
26 -- FF: 17.71 LSTM: 13.77 Hybrid: 14.95 CL-WR: 6.77
27 -- FF: 17.78 LSTM: 13.64 Hybrid: 14.88 CL-WR: 6.8
28 -- FF: 17.86 LSTM: 13.89 Hybrid: 15.09 CL-WR: 6.77
29 -- FF: 18.45 LSTM: 14.12 Hybrid: 15.41 CL-WR: 6.84
30 -- FF: 18.11 LSTM: 13.93 Hybrid: 15.2 CL-WR: 6.85
31 -- FF: 18.56 LSTM: 14.44 Hybrid: 15.67 CL-WR: 6.79
32 -- FF: 18.39 LSTM: 14.45 Hybrid: 15.58 CL-WR: 6.77
33 -- FF: 18.29 LSTM: 14.4 Hybrid: 15.53 CL-WR: 6.77
34 -- FF: 18.24 LSTM: 14.43 Hybrid: 15.5 CL-WR: 6.6
35 -- FF: 19.06 LSTM: 14.39 Hybrid: 15.83 CL-WR: 6.64
36 -- FF: 17.53 LSTM: 13.88 Hybrid: 15.01 CL-WR: 6.59
37 -- FF: 16.59 LSTM: 13.75 Hybrid: 14.58 CL-WR: 6.83
38 -- FF: 17.09 LSTM: 13.44 Hybrid: 14.47 CL-WR: 7.2
39 -- FF: 16.5 LSTM: 13.07 Hybrid: 14.1 CL-WR: 7.2
40 -- FF: 16.19 LSTM: 13.17 Hybrid: 14.0 CL-WR: 7.56
41 -- FF: 16.61 LSTM: 12.92 Hybrid: 13.97 CL-WR: 7.43
42 -- FF: 15.99 LSTM: 12.82 Hybrid: 13.71 CL-WR: 7.57
43 -- FF: 16.0 LSTM: 12.95 Hybrid: 13.77 CL-WR: 7.41
44 -- FF: 17.38 LSTM: 13.51 Hybrid: 14.61 CL-WR: 7.15
45 -- FF: 18.04 LSTM: 13.69 Hybrid: 15.0 CL-WR: 7.16
46 -- FF: 16.3 LSTM: 13.29 Hybrid: 14.19 CL-WR: 6.9
47 -- FF: 15.81 LSTM: 13.02 Hybrid: 13.83 CL-WR: 7.3
48 -- FF: 16.44 LSTM: 13.44 Hybrid: 14.3 CL-WR: 6.87
49 -- FF: 16.6 LSTM: 13.58 Hybrid: 14.44 CL-WR: 6.99
50 -- FF: 17.61 LSTM: 13.43 Hybrid: 14.72 CL-WR: 6.94
51 -- FF: 16.78 LSTM: 13.05 Hybrid: 14.13 CL-WR: 7.39
52 -- FF: 16.33 LSTM: 12.82 Hybrid: 13.77 CL-WR: 7.56
53 -- FF: 16.18 LSTM: 12.63 Hybrid: 13.5 CL-WR: 7.94
54 -- FF: 16.81 LSTM: 12.82 Hybrid: 13.9 CL-WR: 7.84
55 -- FF: 16.74 LSTM: 13.22 Hybrid: 14.25 CL-WR: 6.77
56 -- FF: 17.58 LSTM: 13.53 Hybrid: 14.76 CL-WR: 6.64
57 -- FF: 17.7 LSTM: 13.73 Hybrid: 14.89 CL-WR: 6.64
58 -- FF: 16.94 LSTM: 13.13 Hybrid: 14.28 CL-WR: 6.59
59 -- FF: 17.9 LSTM: 13.97 Hybrid: 15.17 CL-WR: 6.49
60 -- FF: 17.07 LSTM: 13.43 Hybrid: 14.5 CL-WR: 6.6
61 -- FF: 17.09 LSTM: 13.8 Hybrid: 14.76 CL-WR: 6.48
62 -- FF: 17.23 LSTM: 13.07 Hybrid: 14.26 CL-WR: 7.18
63 -- FF: 18.71 LSTM: 13.66 Hybrid: 15.13 CL-WR: 6.99
64 -- FF: 17.28 LSTM: 13.14 Hybrid: 14.36 CL-WR: 7.08
65 -- FF: 16.27 LSTM: 13.25 Hybrid: 14.06 CL-WR: 7.12
66 -- FF: 18.2 LSTM: 13.58 Hybrid: 14.99 CL-WR: 6.83
67 -- FF: 16.43 LSTM: 12.96 Hybrid: 14.0 CL-WR: 6.94
68 -- FF: 16.26 LSTM: 13.21 Hybrid: 14.1 CL-WR: 6.68
69 -- FF: 17.3 LSTM: 13.46 Hybrid: 14.55 CL-WR: 6.83
70 -- FF: 17.16 LSTM: 13.5 Hybrid: 14.6 CL-WR: 6.68
71 -- FF: 17.17 LSTM: 13.46 Hybrid: 14.56 CL-WR: 6.64
72 -- FF: 16.77 LSTM: 13.38 Hybrid: 14.36 CL-WR: 6.71
73 -- FF: 17.5 LSTM: 13.48 Hybrid: 14.68 CL-WR: 6.78
74 -- FF: 16.65 LSTM: 13.82 Hybrid: 14.64 CL-WR: 6.51
75 -- FF: 16.3 LSTM: 13.33 Hybrid: 14.15 CL-WR: 6.92
76 -- FF: 16.54 LSTM: 12.79 Hybrid: 13.87 CL-WR: 7.36
77 -- FF: 16.7 LSTM: 13.0 Hybrid: 14.0 CL-WR: 7.46
78 -- FF: 16.05 LSTM: 12.62 Hybrid: 13.56 CL-WR: 7.48
79 -- FF: 16.72 LSTM: 12.95 Hybrid: 14.05 CL-WR: 7.19
80 -- FF: 16.0 LSTM: 13.19 Hybrid: 14.02 CL-WR: 6.75
81 -- FF: 17.46 LSTM: 13.94 Hybrid: 14.98 CL-WR: 6.51
82 -- FF: 16.85 LSTM: 13.55 Hybrid: 14.55 CL-WR: 6.42
83 -- FF: 17.37 LSTM: 13.62 Hybrid: 14.77 CL-WR: 6.54
84 -- FF: 16.74 LSTM: 13.67 Hybrid: 14.6 CL-WR: 6.65
85 -- FF: 17.45 LSTM: 13.74 Hybrid: 14.84 CL-WR: 6.46
86 -- FF: 17.13 LSTM: 13.58 Hybrid: 14.58 CL-WR: 6.78
87 -- FF: 16.74 LSTM: 12.49 Hybrid: 13.66 CL-WR: 7.73
88 -- FF: 17.38 LSTM: 12.65 Hybrid: 13.92 CL-WR: 8.28
89 -- FF: 19.29 LSTM: 12.89 Hybrid: 14.74 CL-WR: 7.8
90 -- FF: 16.7 LSTM: 12.21 Hybrid: 13.44 CL-WR: 8.14
91 -- FF: 18.4 LSTM: 12.69 Hybrid: 14.35 CL-WR: 7.52
92 -- FF: 18.03 LSTM: 12.71 Hybrid: 14.21 CL-WR: 8.11
93 -- FF: 17.07 LSTM: 12.51 Hybrid: 13.75 CL-WR: 7.94
94 -- FF: 17.62 LSTM: 12.76 Hybrid: 14.12 CL-WR: 7.76
95 -- FF: 16.7 LSTM: 12.56 Hybrid: 13.68 CL-WR: 7.61
96 -- FF: 16.94 LSTM: 12.95 Hybrid: 14.09 CL-WR: 7.2
97 -- FF: 17.6 LSTM: 13.0 Hybrid: 14.38 CL-WR: 7.13
98 -- FF: 17.81 LSTM: 13.49 Hybrid: 14.77 CL-WR: 6.74
99 -- FF: 16.72 LSTM: 13.61 Hybrid: 14.49 CL-WR: 6.48
100 -- FF: 16.9 LSTM: 13.13 Hybrid: 14.24 CL-WR: 6.92
101 -- FF: 17.25 LSTM: 12.97 Hybrid: 14.26 CL-WR: 6.84
102 -- FF: 16.92 LSTM: 13.04 Hybrid: 14.2 CL-WR: 6.78
103 -- FF: 17.69 LSTM: 13.49 Hybrid: 14.78 CL-WR: 6.64
104 -- FF: 17.61 LSTM: 13.35 Hybrid: 14.7 CL-WR: 6.58
105 -- FF: 16.45 LSTM: 13.05 Hybrid: 14.07 CL-WR: 6.6
106 -- FF: 16.46 LSTM: 13.19 Hybrid: 14.13 CL-WR: 6.85
107 -- FF: 15.95 LSTM: 12.92 Hybrid: 13.87 CL-WR: 6.45
108 -- FF: 16.12 LSTM: 13.22 Hybrid: 14.12 CL-WR: 6.44
109 -- FF: 16.88 LSTM: 13.35 Hybrid: 14.38 CL-WR: 6.53
110 -- FF: 18.28 LSTM: 13.47 Hybrid: 14.93 CL-WR: 6.76
111 -- FF: 17.69 LSTM: 13.11 Hybrid: 14.49 CL-WR: 7.03
112 -- FF: 17.91 LSTM: 13.24 Hybrid: 14.61 CL-WR: 6.97
113 -- FF: 16.73 LSTM: 12.57 Hybrid: 13.8 CL-WR: 7.35
114 -- FF: 17.18 LSTM: 12.86 Hybrid: 14.12 CL-WR: 7.13
115 -- FF: 17.73 LSTM: 12.69 Hybrid: 14.18 CL-WR: 7.39
116 -- FF: 17.7 LSTM: 12.95 Hybrid: 14.37 CL-WR: 6.84
117 -- FF: 17.15 LSTM: 13.04 Hybrid: 14.21 CL-WR: 6.97
118 -- FF: 17.47 LSTM: 12.78 Hybrid: 14.16 CL-WR: 7.22
119 -- FF: 16.38 LSTM: 12.53 Hybrid: 13.51 CL-WR: 8.02
120 -- FF: 16.22 LSTM: 12.38 Hybrid: 13.36 CL-WR: 8.1
121 -- FF: 16.78 LSTM: 12.32 Hybrid: 13.56 CL-WR: 7.8
122 -- FF: 16.55 LSTM: 12.41 Hybrid: 13.58 CL-WR: 7.41
123 -- FF: 16.23 LSTM: 12.85 Hybrid: 13.74 CL-WR: 7.21
124 -- FF: 17.26 LSTM: 13.22 Hybrid: 14.44 CL-WR: 6.53
125 -- FF: 17.23 LSTM: 12.86 Hybrid: 14.11 CL-WR: 7.02
126 -- FF: 18.35 LSTM: 12.72 Hybrid: 14.34 CL-WR: 7.2
127 -- FF: 17.36 LSTM: 12.67 Hybrid: 14.07 CL-WR: 7.07
128 -- FF: 19.21 LSTM: 12.9 Hybrid: 14.85 CL-WR: 7.02
129 -- FF: 17.32 LSTM: 13.06 Hybrid: 14.33 CL-WR: 6.6
130 -- FF: 18.66 LSTM: 13.7 Hybrid: 15.3 CL-WR: 6.22
131 -- FF: 18.43 LSTM: 13.77 Hybrid: 15.25 CL-WR: 6.1
132 -- FF: 18.66 LSTM: 13.84 Hybrid: 15.37 CL-WR: 6.25
133 -- FF: 18.47 LSTM: 13.78 Hybrid: 15.22 CL-WR: 6.29
134 -- FF: 17.61 LSTM: 13.67 Hybrid: 14.86 CL-WR: 6.13
135 -- FF: 17.57 LSTM: 13.31 Hybrid: 14.65 CL-WR: 6.18
136 -- FF: 17.2 LSTM: 12.58 Hybrid: 13.97 CL-WR: 6.75
137 -- FF: 17.37 LSTM: 12.7 Hybrid: 14.11 CL-WR: 6.98
138 -- FF: 18.4 LSTM: 12.89 Hybrid: 14.58 CL-WR: 6.88
139 -- FF: 19.1 LSTM: 13.02 Hybrid: 14.95 CL-WR: 6.79
140 -- FF: 18.13 LSTM: 12.82 Hybrid: 14.39 CL-WR: 7.04
141 -- FF: 17.82 LSTM: 13.03 Hybrid: 14.48 CL-WR: 6.53
142 -- FF: 16.92 LSTM: 12.77 Hybrid: 14.0 CL-WR: 6.58
143 -- FF: 16.52 LSTM: 12.77 Hybrid: 13.9 CL-WR: 6.54
144 -- FF: 16.83 LSTM: 12.41 Hybrid: 13.79 CL-WR: 6.8
145 -- FF: 15.82 LSTM: 12.52 Hybrid: 13.39 CL-WR: 7.29
146 -- FF: 17.8 LSTM: 12.3 Hybrid: 13.83 CL-WR: 8.06
147 -- FF: 17.81 LSTM: 12.29 Hybrid: 13.88 CL-WR: 7.62
148 -- FF: 18.45 LSTM: 12.76 Hybrid: 14.45 CL-WR: 7.2
149 -- FF: 18.42 LSTM: 12.69 Hybrid: 14.42 CL-WR: 7.21
150 -- FF: 17.99 LSTM: 12.6 Hybrid: 14.25 CL-WR: 6.94
151 -- FF: 17.76 LSTM: 12.92 Hybrid: 14.35 CL-WR: 6.85
152 -- FF: 17.32 LSTM: 12.77 Hybrid: 14.17 CL-WR: 6.67
153 -- FF: 17.11 LSTM: 12.62 Hybrid: 13.96 CL-WR: 6.95
154 -- FF: 18.25 LSTM: 13.58 Hybrid: 15.06 CL-WR: 6.3
155 -- FF: 17.99 LSTM: 13.57 Hybrid: 14.97 CL-WR: 6.13
156 -- FF: 17.39 LSTM: 13.16 Hybrid: 14.53 CL-WR: 6.13
157 -- FF: 18.18 LSTM: 13.48 Hybrid: 14.98 CL-WR: 6.16
158 -- FF: 17.25 LSTM: 13.15 Hybrid: 14.45 CL-WR: 6.14
159 -- FF: 17.33 LSTM: 13.33 Hybrid: 14.59 CL-WR: 6.2
160 -- FF: 17.22 LSTM: 13.12 Hybrid: 14.35 CL-WR: 6.36
161 -- FF: 16.48 LSTM: 12.39 Hybrid: 13.58 CL-WR: 6.99
162 -- FF: 16.24 LSTM: 12.22 Hybrid: 13.32 CL-WR: 7.63
163 -- FF: 17.51 LSTM: 12.35 Hybrid: 13.8 CL-WR: 7.7
164 -- FF: 17.05 LSTM: 12.3 Hybrid: 13.63 CL-WR: 7.43
165 -- FF: 17.13 LSTM: 12.34 Hybrid: 13.7 CL-WR: 7.24
166 -- FF: 17.42 LSTM: 12.47 Hybrid: 13.94 CL-WR: 7.16
167 -- FF: 16.54 LSTM: 12.19 Hybrid: 13.42 CL-WR: 7.29
168 -- FF: 16.31 LSTM: 12.4 Hybrid: 13.5 CL-WR: 7.23
169 -- FF: 18.13 LSTM: 12.55 Hybrid: 14.17 CL-WR: 7.31
170 -- FF: 16.14 LSTM: 12.31 Hybrid: 13.38 CL-WR: 7.26
171 -- FF: 16.42 LSTM: 12.32 Hybrid: 13.44 CL-WR: 7.44
172 -- FF: 16.98 LSTM: 12.52 Hybrid: 13.81 CL-WR: 7.11
173 -- FF: 17.28 LSTM: 12.37 Hybrid: 13.75 CL-WR: 7.39
174 -- FF: 17.23 LSTM: 12.58 Hybrid: 13.97 CL-WR: 6.82
175 -- FF: 16.72 LSTM: 12.51 Hybrid: 13.68 CL-WR: 7.18
176 -- FF: 18.02 LSTM: 12.84 Hybrid: 14.36 CL-WR: 6.72
177 -- FF: 17.11 LSTM: 12.43 Hybrid: 13.81 CL-WR: 6.83
178 -- FF: 16.61 LSTM: 12.28 Hybrid: 13.62 CL-WR: 6.71
179 -- FF: 16.13 LSTM: 12.72 Hybrid: 13.76 CL-WR: 6.36
180 -- FF: 15.98 LSTM: 12.83 Hybrid: 13.86 CL-WR: 5.96
181 -- FF: 16.04 LSTM: 13.03 Hybrid: 13.97 CL-WR: 5.96
182 -- FF: 16.81 LSTM: 13.23 Hybrid: 14.31 CL-WR: 5.97
183 -- FF: 16.95 LSTM: 13.02 Hybrid: 14.18 CL-WR: 6.34
184 -- FF: 16.0 LSTM: 12.58 Hybrid: 13.61 CL-WR: 6.46
185 -- FF: 16.63 LSTM: 13.04 Hybrid: 14.11 CL-WR: 6.34
186 -- FF: 17.14 LSTM: 12.54 Hybrid: 13.97 CL-WR: 6.58
187 -- FF: 16.55 LSTM: 12.18 Hybrid: 13.38 CL-WR: 7.43
188 -- FF: 16.78 LSTM: 12.06 Hybrid: 13.3 CL-WR: 8.31
189 -- FF: 17.32 LSTM: 12.09 Hybrid: 13.6 CL-WR: 7.52
190 -- FF: 18.08 LSTM: 12.23 Hybrid: 13.94 CL-WR: 7.61
191 -- FF: 17.73 LSTM: 12.29 Hybrid: 13.9 CL-WR: 7.08
192 -- FF: 16.67 LSTM: 12.36 Hybrid: 13.58 CL-WR: 6.86
193 -- FF: 16.61 LSTM: 12.29 Hybrid: 13.55 CL-WR: 7.01
194 -- FF: 16.37 LSTM: 12.08 Hybrid: 13.2 CL-WR: 7.92
195 -- FF: 16.33 LSTM: 12.24 Hybrid: 13.38 CL-WR: 7.23
196 -- FF: 17.88 LSTM: 12.55 Hybrid: 14.12 CL-WR: 6.83
197 -- FF: 17.11 LSTM: 12.75 Hybrid: 13.99 CL-WR: 6.93
198 -- FF: 16.53 LSTM: 12.32 Hybrid: 13.56 CL-WR: 6.73
199 -- FF: 15.13 LSTM: 12.37 Hybrid: 13.12 CL-WR: 6.68
200 -- FF: 16.27 LSTM: 12.29 Hybrid: 13.42 CL-WR: 6.84
201 -- FF: 15.55 LSTM: 11.93 Hybrid: 12.9 CL-WR: 7.47
202 -- FF: 16.93 LSTM: 12.42 Hybrid: 13.77 CL-WR: 6.7
203 -- FF: 15.82 LSTM: 12.09 Hybrid: 13.12 CL-WR: 7.15
204 -- FF: 15.79 LSTM: 12.26 Hybrid: 13.24 CL-WR: 7.04
205 -- FF: 15.77 LSTM: 12.39 Hybrid: 13.36 CL-WR: 6.74
206 -- FF: 16.28 LSTM: 12.67 Hybrid: 13.75 CL-WR: 6.5
207 -- FF: 16.41 LSTM: 12.71 Hybrid: 13.8 CL-WR: 6.55
208 -- FF: 15.68 LSTM: 12.56 Hybrid: 13.42 CL-WR: 6.8
209 -- FF: 16.33 LSTM: 12.87 Hybrid: 13.86 CL-WR: 6.42
210 -- FF: 16.53 LSTM: 12.75 Hybrid: 13.89 CL-WR: 6.23
211 -- FF: 15.13 LSTM: 12.44 Hybrid: 13.18 CL-WR: 6.71
212 -- FF: 16.71 LSTM: 12.12 Hybrid: 13.45 CL-WR: 7.15
213 -- FF: 17.27 LSTM: 12.48 Hybrid: 13.93 CL-WR: 6.6
214 -- FF: 16.17 LSTM: 12.24 Hybrid: 13.33 CL-WR: 6.99
215 -- FF: 16.37 LSTM: 12.55 Hybrid: 13.71 CL-WR: 6.41
216 -- FF: 15.83 LSTM: 12.17 Hybrid: 13.24 CL-WR: 6.62
217 -- FF: 15.45 LSTM: 11.86 Hybrid: 12.89 CL-WR: 6.98
218 -- FF: 16.37 LSTM: 12.03 Hybrid: 13.25 CL-WR: 7.2
219 -- FF: 16.93 LSTM: 12.3 Hybrid: 13.65 CL-WR: 6.97
220 -- FF: 16.21 LSTM: 12.14 Hybrid: 13.29 CL-WR: 7.12
221 -- FF: 16.88 LSTM: 12.24 Hybrid: 13.58 CL-WR: 7.03
222 -- FF: 16.27 LSTM: 11.88 Hybrid: 13.07 CL-WR: 7.63
223 -- FF: 16.54 LSTM: 12.04 Hybrid: 13.28 CL-WR: 7.66
224 -- FF: 15.76 LSTM: 12.1 Hybrid: 13.15 CL-WR: 6.77
225 -- FF: 16.05 LSTM: 12.15 Hybrid: 13.27 CL-WR: 7.02
226 -- FF: 16.89 LSTM: 12.27 Hybrid: 13.6 CL-WR: 6.73
227 -- FF: 17.28 LSTM: 12.23 Hybrid: 13.75 CL-WR: 6.63
228 -- FF: 16.29 LSTM: 12.34 Hybrid: 13.44 CL-WR: 6.87
229 -- FF: 15.9 LSTM: 12.02 Hybrid: 13.08 CL-WR: 7.05
230 -- FF: 15.01 LSTM: 12.03 Hybrid: 12.85 CL-WR: 6.87
231 -- FF: 16.85 LSTM: 12.73 Hybrid: 13.97 CL-WR: 6.29
232 -- FF: 15.73 LSTM: 12.64 Hybrid: 13.56 CL-WR: 6.12
233 -- FF: 16.53 LSTM: 12.79 Hybrid: 13.96 CL-WR: 5.92
234 -- FF: 16.21 LSTM: 12.45 Hybrid: 13.55 CL-WR: 6.37
235 -- FF: 15.9 LSTM: 12.37 Hybrid: 13.41 CL-WR: 6.52
236 -- FF: 15.9 LSTM: 12.27 Hybrid: 13.33 CL-WR: 6.63
237 -- FF: 16.96 LSTM: 12.37 Hybrid: 13.71 CL-WR: 6.79
238 -- FF: 15.98 LSTM: 11.9 Hybrid: 12.99 CL-WR: 7.64
239 -- FF: 16.95 LSTM: 11.86 Hybrid: 13.27 CL-WR: 7.6
240 -- FF: 17.29 LSTM: 11.84 Hybrid: 13.37 CL-WR: 7.65
241 -- FF: 16.63 LSTM: 11.7 Hybrid: 13.04 CL-WR: 8.02
242 -- FF: 17.14 LSTM: 11.8 Hybrid: 13.29 CL-WR: 7.78
243 -- FF: 17.33 LSTM: 11.91 Hybrid: 13.49 CL-WR: 7.55
244 -- FF: 15.9 LSTM: 11.88 Hybrid: 12.99 CL-WR: 6.92
245 -- FF: 15.96 LSTM: 11.97 Hybrid: 13.12 CL-WR: 6.87
246 -- FF: 16.21 LSTM: 12.13 Hybrid: 13.26 CL-WR: 6.97
247 -- FF: 17.35 LSTM: 12.02 Hybrid: 13.71 CL-WR: 6.37
248 -- FF: 18.15 LSTM: 12.28 Hybrid: 14.18 CL-WR: 6.23
249 -- FF: 17.78 LSTM: 12.84 Hybrid: 14.44 CL-WR: 5.94
250 -- FF: 17.47 LSTM: 12.65 Hybrid: 14.21 CL-WR: 5.99
251 -- FF: 16.29 LSTM: 12.06 Hybrid: 13.37 CL-WR: 6.51
252 -- FF: 17.13 LSTM: 12.36 Hybrid: 13.83 CL-WR: 6.81
253 -- FF: 16.55 LSTM: 12.02 Hybrid: 13.38 CL-WR: 6.78
254 -- FF: 14.68 LSTM: 11.93 Hybrid: 12.55 CL-WR: 7.47
255 -- FF: 15.54 LSTM: 11.94 Hybrid: 12.91 CL-WR: 6.88
256 -- FF: 15.04 LSTM: 11.94 Hybrid: 12.8 CL-WR: 6.64
257 -- FF: 15.64 LSTM: 12.31 Hybrid: 13.25 CL-WR: 6.49
258 -- FF: 16.92 LSTM: 11.89 Hybrid: 13.39 CL-WR: 6.96
259 -- FF: 16.4 LSTM: 12.36 Hybrid: 13.56 CL-WR: 6.37
260 -- FF: 15.18 LSTM: 11.98 Hybrid: 12.87 CL-WR: 6.57
261 -- FF: 14.76 LSTM: 11.88 Hybrid: 12.67 CL-WR: 6.75
262 -- FF: 15.64 LSTM: 11.88 Hybrid: 12.88 CL-WR: 7.46
263 -- FF: 14.63 LSTM: 11.87 Hybrid: 12.59 CL-WR: 6.84
264 -- FF: 15.98 LSTM: 11.94 Hybrid: 13.17 CL-WR: 6.43
265 -- FF: 15.36 LSTM: 12.0 Hybrid: 12.96 CL-WR: 6.31
266 -- FF: 15.48 LSTM: 12.51 Hybrid: 13.44 CL-WR: 5.83
267 -- FF: 15.32 LSTM: 12.3 Hybrid: 13.22 CL-WR: 5.8
268 -- FF: 16.21 LSTM: 12.84 Hybrid: 13.85 CL-WR: 6.01
269 -- FF: 15.15 LSTM: 12.29 Hybrid: 13.14 CL-WR: 5.98
270 -- FF: 14.99 LSTM: 12.03 Hybrid: 12.86 CL-WR: 6.51
271 -- FF: 14.67 LSTM: 11.9 Hybrid: 12.55 CL-WR: 7.16
272 -- FF: 14.74 LSTM: 11.99 Hybrid: 12.67 CL-WR: 6.89
273 -- FF: 14.97 LSTM: 12.05 Hybrid: 12.9 CL-WR: 6.34
274 -- FF: 15.78 LSTM: 12.55 Hybrid: 13.49 CL-WR: 5.98
275 -- FF: 16.39 LSTM: 12.48 Hybrid: 13.67 CL-WR: 6.01
276 -- FF: 15.89 LSTM: 12.49 Hybrid: 13.51 CL-WR: 5.88
277 -- FF: 16.51 LSTM: 12.31 Hybrid: 13.59 CL-WR: 6.09
278 -- FF: 16.05 LSTM: 12.06 Hybrid: 13.29 CL-WR: 6.2
279 -- FF: 16.22 LSTM: 12.37 Hybrid: 13.55 CL-WR: 5.94
280 -- FF: 16.21 LSTM: 11.9 Hybrid: 13.09 CL-WR: 6.97
281 -- FF: 16.22 LSTM: 11.48 Hybrid: 12.82 CL-WR: 7.13
282 -- FF: 16.68 LSTM: 11.69 Hybrid: 13.08 CL-WR: 7.43
283 -- FF: 16.0 LSTM: 12.13 Hybrid: 13.28 CL-WR: 6.32
284 -- FF: 17.12 LSTM: 12.1 Hybrid: 13.7 CL-WR: 6.23
285 -- FF: 16.76 LSTM: 12.49 Hybrid: 13.76 CL-WR: 6.17
286 -- FF: 16.28 LSTM: 12.12 Hybrid: 13.41 CL-WR: 6.19
287 -- FF: 16.71 LSTM: 11.88 Hybrid: 13.36 CL-WR: 6.6
288 -- FF: 16.07 LSTM: 11.8 Hybrid: 13.03 CL-WR: 6.91
289 -- FF: 15.7 LSTM: 11.79 Hybrid: 12.99 CL-WR: 6.5
290 -- FF: 16.03 LSTM: 11.96 Hybrid: 13.2 CL-WR: 6.43
291 -- FF: 14.77 LSTM: 11.92 Hybrid: 12.68 CL-WR: 6.53
292 -- FF: 15.2 LSTM: 11.83 Hybrid: 12.74 CL-WR: 6.65
293 -- FF: 16.08 LSTM: 11.99 Hybrid: 13.27 CL-WR: 6.13
294 -- FF: 16.32 LSTM: 11.74 Hybrid: 13.02 CL-WR: 6.88
295 -- FF: 16.42 LSTM: 11.74 Hybrid: 13.08 CL-WR: 6.95
296 -- FF: 16.36 LSTM: 11.53 Hybrid: 12.95 CL-WR: 6.89
297 -- FF: 17.06 LSTM: 11.72 Hybrid: 13.28 CL-WR: 7.2
298 -- FF: 17.08 LSTM: 12.13 Hybrid: 13.61 CL-WR: 6.61
299 -- FF: 15.74 LSTM: 12.12 Hybrid: 13.23 CL-WR: 5.94
300 -- FF: 16.65 LSTM: 12.59 Hybrid: 13.87 CL-WR: 5.65
301 -- FF: 16.55 LSTM: 12.39 Hybrid: 13.69 CL-WR: 5.93
302 -- FF: 17.56 LSTM: 12.42 Hybrid: 14.0 CL-WR: 6.33
303 -- FF: 16.77 LSTM: 12.04 Hybrid: 13.51 CL-WR: 6.05
304 -- FF: 16.43 LSTM: 12.03 Hybrid: 13.45 CL-WR: 5.83
305 -- FF: 15.93 LSTM: 12.15 Hybrid: 13.25 CL-WR: 6.14
306 -- FF: 15.76 LSTM: 11.64 Hybrid: 12.81 CL-WR: 7.01
307 -- FF: 16.8 LSTM: 11.5 Hybrid: 13.06 CL-WR: 7.23
308 -- FF: 16.36 LSTM: 11.76 Hybrid: 13.06 CL-WR: 7.09
309 -- FF: 15.68 LSTM: 11.62 Hybrid: 12.73 CL-WR: 6.99
310 -- FF: 16.61 LSTM: 11.63 Hybrid: 13.01 CL-WR: 7.18
311 -- FF: 16.18 LSTM: 11.84 Hybrid: 13.04 CL-WR: 6.88
312 -- FF: 15.57 LSTM: 11.79 Hybrid: 12.87 CL-WR: 6.41
313 -- FF: 17.38 LSTM: 12.02 Hybrid: 13.65 CL-WR: 6.62
314 -- FF: 16.55 LSTM: 11.84 Hybrid: 13.26 CL-WR: 6.29
315 -- FF: 16.11 LSTM: 11.78 Hybrid: 13.13 CL-WR: 6.11
316 -- FF: 17.02 LSTM: 11.99 Hybrid: 13.53 CL-WR: 6.43
317 -- FF: 15.44 LSTM: 11.9 Hybrid: 12.91 CL-WR: 6.36
318 -- FF: 17.45 LSTM: 12.31 Hybrid: 14.0 CL-WR: 5.78
319 -- FF: 16.88 LSTM: 12.17 Hybrid: 13.67 CL-WR: 5.81
320 -- FF: 15.87 LSTM: 11.87 Hybrid: 13.08 CL-WR: 6.24
321 -- FF: 16.65 LSTM: 11.82 Hybrid: 13.17 CL-WR: 6.97
322 -- FF: 16.87 LSTM: 11.75 Hybrid: 13.3 CL-WR: 6.59
323 -- FF: 16.16 LSTM: 11.75 Hybrid: 13.01 CL-WR: 6.65
324 -- FF: 16.6 LSTM: 11.93 Hybrid: 13.41 CL-WR: 6.03
325 -- FF: 16.74 LSTM: 12.0 Hybrid: 13.44 CL-WR: 6.1
326 -- FF: 15.97 LSTM: 11.84 Hybrid: 13.1 CL-WR: 6.05
327 -- FF: 15.88 LSTM: 11.9 Hybrid: 13.12 CL-WR: 6.01
328 -- FF: 16.17 LSTM: 12.02 Hybrid: 13.31 CL-WR: 6.14
329 -- FF: 16.29 LSTM: 11.92 Hybrid: 13.31 CL-WR: 5.72
330 -- FF: 16.24 LSTM: 12.07 Hybrid: 13.36 CL-WR: 5.89
331 -- FF: 17.03 LSTM: 11.72 Hybrid: 13.25 CL-WR: 6.79
332 -- FF: 17.47 LSTM: 11.6 Hybrid: 13.42 CL-WR: 6.42
333 -- FF: 16.07 LSTM: 11.51 Hybrid: 12.81 CL-WR: 6.69
334 -- FF: 16.82 LSTM: 11.78 Hybrid: 13.3 CL-WR: 6.36
335 -- FF: 15.51 LSTM: 11.47 Hybrid: 12.55 CL-WR: 6.84
336 -- FF: 15.76 LSTM: 11.34 Hybrid: 12.54 CL-WR: 7.2
337 -- FF: 15.54 LSTM: 11.59 Hybrid: 12.78 CL-WR: 6.17
338 -- FF: 15.65 LSTM: 11.59 Hybrid: 12.75 CL-WR: 6.57
339 -- FF: 16.6 LSTM: 11.92 Hybrid: 13.39 CL-WR: 6.0
340 -- FF: 15.49 LSTM: 11.63 Hybrid: 12.67 CL-WR: 6.58
341 -- FF: 15.58 LSTM: 11.68 Hybrid: 12.75 CL-WR: 6.55
342 -- FF: 16.65 LSTM: 11.93 Hybrid: 13.38 CL-WR: 6.07
343 -- FF: 15.3 LSTM: 11.41 Hybrid: 12.57 CL-WR: 6.42
344 -- FF: 15.82 LSTM: 11.53 Hybrid: 12.86 CL-WR: 6.2
345 -- FF: 15.86 LSTM: 11.52 Hybrid: 12.75 CL-WR: 6.68
346 -- FF: 15.94 LSTM: 11.59 Hybrid: 12.89 CL-WR: 6.34
347 -- FF: 15.93 LSTM: 11.59 Hybrid: 12.87 CL-WR: 6.58
348 -- FF: 15.58 LSTM: 11.52 Hybrid: 12.74 CL-WR: 6.47
349 -- FF: 15.95 LSTM: 11.53 Hybrid: 12.85 CL-WR: 6.49
350 -- FF: 15.71 LSTM: 11.69 Hybrid: 12.88 CL-WR: 6.36
351 -- FF: 15.27 LSTM: 11.61 Hybrid: 12.75 CL-WR: 6.12
352 -- FF: 16.63 LSTM: 11.89 Hybrid: 13.43 CL-WR: 5.87
353 -- FF: 15.98 LSTM: 11.47 Hybrid: 12.89 CL-WR: 6.12
354 -- FF: 15.94 LSTM: 11.77 Hybrid: 13.1 CL-WR: 5.77
355 -- FF: 16.34 LSTM: 11.7 Hybrid: 13.16 CL-WR: 5.81
356 -- FF: 16.84 LSTM: 11.96 Hybrid: 13.57 CL-WR: 5.65
357 -- FF: 16.38 LSTM: 11.51 Hybrid: 13.06 CL-WR: 5.99
358 -- FF: 15.88 LSTM: 11.48 Hybrid: 12.75 CL-WR: 6.24
359 -- FF: 16.55 LSTM: 11.41 Hybrid: 12.9 CL-WR: 6.42
360 -- FF: 16.27 LSTM: 11.57 Hybrid: 12.96 CL-WR: 6.15
361 -- FF: 15.84 LSTM: 11.4 Hybrid: 12.71 CL-WR: 6.13
362 -- FF: 16.25 LSTM: 11.54 Hybrid: 12.94 CL-WR: 6.09
363 -- FF: 15.67 LSTM: 11.41 Hybrid: 12.51 CL-WR: 6.62
364 -- FF: 15.07 LSTM: 11.38 Hybrid: 12.22 CL-WR: 7.4
365 -- FF: 15.38 LSTM: 11.04 Hybrid: 12.16 CL-WR: 7.34
366 -- FF: 15.85 LSTM: 11.44 Hybrid: 12.61 CL-WR: 6.61
367 -- FF: 17.92 LSTM: 11.34 Hybrid: 13.37 CL-WR: 6.38
368 -- FF: 16.61 LSTM: 11.15 Hybrid: 12.55 CL-WR: 7.35
369 -- FF: 18.45 LSTM: 10.98 Hybrid: 13.22 CL-WR: 6.86
370 -- FF: 15.42 LSTM: 11.33 Hybrid: 12.48 CL-WR: 6.27
371 -- FF: 16.15 LSTM: 11.45 Hybrid: 12.87 CL-WR: 6.1
372 -- FF: 15.61 LSTM: 11.15 Hybrid: 12.41 CL-WR: 6.33
373 -- FF: 17.24 LSTM: 11.25 Hybrid: 12.99 CL-WR: 6.61
374 -- FF: 17.67 LSTM: 11.27 Hybrid: 13.24 CL-WR: 6.41
375 -- FF: 17.0 LSTM: 11.5 Hybrid: 13.22 CL-WR: 6.11
376 -- FF: 16.43 LSTM: 11.69 Hybrid: 13.19 CL-WR: 5.87
377 -- FF: 16.55 LSTM: 11.68 Hybrid: 13.24 CL-WR: 5.82
378 -- FF: 16.18 LSTM: 11.46 Hybrid: 12.93 CL-WR: 5.98
379 -- FF: 16.34 LSTM: 11.81 Hybrid: 13.26 CL-WR: 5.78
380 -- FF: 15.92 LSTM: 11.53 Hybrid: 12.85 CL-WR: 5.87
381 -- FF: 16.07 LSTM: 11.39 Hybrid: 12.85 CL-WR: 5.87
382 -- FF: 18.04 LSTM: 11.71 Hybrid: 13.76 CL-WR: 5.68
383 -- FF: 17.99 LSTM: 11.28 Hybrid: 13.29 CL-WR: 6.4
384 -- FF: 15.71 LSTM: 11.24 Hybrid: 12.5 CL-WR: 6.23
385 -- FF: 15.66 LSTM: 11.23 Hybrid: 12.44 CL-WR: 6.26
386 -- FF: 18.19 LSTM: 11.35 Hybrid: 13.45 CL-WR: 6.13
387 -- FF: 15.97 LSTM: 11.05 Hybrid: 12.42 CL-WR: 6.5
388 -- FF: 18.14 LSTM: 11.28 Hybrid: 13.32 CL-WR: 6.39
389 -- FF: 16.7 LSTM: 11.58 Hybrid: 13.14 CL-WR: 5.75
390 -- FF: 17.99 LSTM: 11.38 Hybrid: 13.5 CL-WR: 5.85
391 -- FF: 15.89 LSTM: 11.19 Hybrid: 12.57 CL-WR: 6.04
392 -- FF: 16.2 LSTM: 11.08 Hybrid: 12.47 CL-WR: 6.47
393 -- FF: 15.98 LSTM: 11.19 Hybrid: 12.59 CL-WR: 6.05
394 -- FF: 17.36 LSTM: 11.13 Hybrid: 12.94 CL-WR: 6.24
395 -- FF: 16.54 LSTM: 11.21 Hybrid: 12.74 CL-WR: 6.23
396 -- FF: 17.23 LSTM: 11.06 Hybrid: 12.94 CL-WR: 6.17
397 -- FF: 17.61 LSTM: 11.4 Hybrid: 13.27 CL-WR: 5.98
398 -- FF: 17.07 LSTM: 11.16 Hybrid: 12.78 CL-WR: 6.71
399 -- FF: 17.59 LSTM: 11.17 Hybrid: 12.96 CL-WR: 6.52
400 -- FF: 17.42 LSTM: 10.98 Hybrid: 12.78 CL-WR: 6.62
401 -- FF: 18.36 LSTM: 11.29 Hybrid: 13.47 CL-WR: 6.15
402 -- FF: 16.57 LSTM: 11.0 Hybrid: 12.74 CL-WR: 5.96
403 -- FF: 17.79 LSTM: 11.42 Hybrid: 13.4 CL-WR: 5.91
404 -- FF: 17.64 LSTM: 11.48 Hybrid: 13.43 CL-WR: 5.87
405 -- FF: 16.99 LSTM: 11.32 Hybrid: 13.1 CL-WR: 5.86
406 -- FF: 16.33 LSTM: 10.99 Hybrid: 12.54 CL-WR: 6.23
407 -- FF: 17.75 LSTM: 11.21 Hybrid: 13.31 CL-WR: 5.79
408 -- FF: 17.77 LSTM: 11.5 Hybrid: 13.42 CL-WR: 5.88
409 -- FF: 18.35 LSTM: 11.24 Hybrid: 13.45 CL-WR: 6.04
410 -- FF: 16.85 LSTM: 10.99 Hybrid: 12.66 CL-WR: 6.18
411 -- FF: 17.06 LSTM: 11.08 Hybrid: 12.85 CL-WR: 5.92
412 -- FF: 17.88 LSTM: 10.73 Hybrid: 12.8 CL-WR: 6.29
413 -- FF: 17.72 LSTM: 10.9 Hybrid: 12.85 CL-WR: 6.36
414 -- FF: 18.23 LSTM: 10.82 Hybrid: 13.02 CL-WR: 6.01
415 -- FF: 17.58 LSTM: 10.86 Hybrid: 12.69 CL-WR: 6.59
416 -- FF: 18.09 LSTM: 10.9 Hybrid: 12.94 CL-WR: 6.54
417 -- FF: 18.32 LSTM: 11.06 Hybrid: 13.23 CL-WR: 6.04
418 -- FF: 18.27 LSTM: 10.87 Hybrid: 13.05 CL-WR: 6.13
419 -- FF: 17.51 LSTM: 10.56 Hybrid: 12.56 CL-WR: 6.33
420 -- FF: 16.97 LSTM: 10.5 Hybrid: 12.19 CL-WR: 7.0
421 -- FF: 18.69 LSTM: 10.93 Hybrid: 13.31 CL-WR: 6.73
422 -- FF: 18.34 LSTM: 11.09 Hybrid: 13.4 CL-WR: 6.4
423 -- FF: 17.66 LSTM: 10.86 Hybrid: 12.93 CL-WR: 6.81
424 -- FF: 17.88 LSTM: 10.79 Hybrid: 12.87 CL-WR: 7.38
425 -- FF: 18.47 LSTM: 11.1 Hybrid: 13.4 CL-WR: 6.73
426 -- FF: 17.83 LSTM: 10.95 Hybrid: 12.95 CL-WR: 7.17
427 -- FF: 17.93 LSTM: 10.89 Hybrid: 13.03 CL-WR: 6.76
428 -- FF: 17.74 LSTM: 10.86 Hybrid: 12.87 CL-WR: 6.9
429 -- FF: 18.39 LSTM: 11.0 Hybrid: 13.28 CL-WR: 6.64
430 -- FF: 18.77 LSTM: 11.11 Hybrid: 13.51 CL-WR: 6.37
431 -- FF: 18.51 LSTM: 11.15 Hybrid: 13.49 CL-WR: 6.47
432 -- FF: 18.23 LSTM: 11.39 Hybrid: 13.54 CL-WR: 6.09
433 -- FF: 18.22 LSTM: 11.23 Hybrid: 13.43 CL-WR: 6.34
434 -- FF: 17.79 LSTM: 11.08 Hybrid: 13.16 CL-WR: 6.29
435 -- FF: 17.35 LSTM: 10.75 Hybrid: 12.59 CL-WR: 7.21
436 -- FF: 18.19 LSTM: 10.91 Hybrid: 13.12 CL-WR: 6.78
437 -- FF: 17.69 LSTM: 10.74 Hybrid: 12.73 CL-WR: 7.12
438 -- FF: 18.35 LSTM: 10.85 Hybrid: 13.14 CL-WR: 6.66
439 -- FF: 17.74 LSTM: 10.75 Hybrid: 12.81 CL-WR: 6.65
440 -- FF: 17.84 LSTM: 10.97 Hybrid: 13.0 CL-WR: 6.54
441 -- FF: 18.45 LSTM: 11.2 Hybrid: 13.33 CL-WR: 6.64
442 -- FF: 18.01 LSTM: 10.75 Hybrid: 12.9 CL-WR: 6.6
443 -- FF: 17.9 LSTM: 10.69 Hybrid: 12.84 CL-WR: 6.7
444 -- FF: 18.66 LSTM: 11.05 Hybrid: 13.35 CL-WR: 6.57
445 -- FF: 18.28 LSTM: 10.93 Hybrid: 13.14 CL-WR: 6.28
446 -- FF: 18.1 LSTM: 11.23 Hybrid: 13.22 CL-WR: 6.17
447 -- FF: 18.63 LSTM: 11.21 Hybrid: 13.48 CL-WR: 6.08
448 -- FF: 18.43 LSTM: 11.11 Hybrid: 13.36 CL-WR: 6.22
449 -- FF: 17.44 LSTM: 10.66 Hybrid: 12.67 CL-WR: 6.35
450 -- FF: 17.1 LSTM: 10.65 Hybrid: 12.5 CL-WR: 6.64
451 -- FF: 18.03 LSTM: 10.62 Hybrid: 12.83 CL-WR: 6.54
452 -- FF: 18.8 LSTM: 11.05 Hybrid: 13.43 CL-WR: 6.45
453 -- FF: 18.58 LSTM: 11.14 Hybrid: 13.39 CL-WR: 6.32
454 -- FF: 18.33 LSTM: 11.16 Hybrid: 13.4 CL-WR: 5.93
455 -- FF: 19.04 LSTM: 11.68 Hybrid: 13.95 CL-WR: 5.86
456 -- FF: 18.01 LSTM: 11.52 Hybrid: 13.43 CL-WR: 5.85
457 -- FF: 18.57 LSTM: 12.0 Hybrid: 13.95 CL-WR: 5.85
458 -- FF: 18.24 LSTM: 11.42 Hybrid: 13.55 CL-WR: 5.75
459 -- FF: 18.15 LSTM: 11.28 Hybrid: 13.41 CL-WR: 5.87
460 -- FF: 17.47 LSTM: 11.03 Hybrid: 12.99 CL-WR: 5.94
461 -- FF: 18.54 LSTM: 11.12 Hybrid: 13.36 CL-WR: 6.31
462 -- FF: 17.67 LSTM: 10.74 Hybrid: 12.71 CL-WR: 6.7
463 -- FF: 17.19 LSTM: 10.51 Hybrid: 12.47 CL-WR: 6.38
464 -- FF: 16.99 LSTM: 11.04 Hybrid: 12.64 CL-WR: 6.64
465 -- FF: 17.28 LSTM: 10.9 Hybrid: 12.81 CL-WR: 6.32
466 -- FF: 17.06 LSTM: 10.83 Hybrid: 12.62 CL-WR: 6.46
467 -- FF: 17.29 LSTM: 10.59 Hybrid: 12.48 CL-WR: 6.88
468 -- FF: 17.08 LSTM: 10.6 Hybrid: 12.34 CL-WR: 7.05
469 -- FF: 16.04 LSTM: 10.43 Hybrid: 11.82 CL-WR: 7.16
470 -- FF: 15.72 LSTM: 10.28 Hybrid: 11.6 CL-WR: 6.98
471 -- FF: 16.06 LSTM: 10.5 Hybrid: 11.99 CL-WR: 6.86
472 -- FF: 16.75 LSTM: 10.84 Hybrid: 12.44 CL-WR: 6.89
473 -- FF: 16.22 LSTM: 10.39 Hybrid: 11.97 CL-WR: 7.08
474 -- FF: 16.77 LSTM: 10.44 Hybrid: 12.23 CL-WR: 6.97
475 -- FF: 17.57 LSTM: 10.58 Hybrid: 12.51 CL-WR: 7.03
476 -- FF: 15.8 LSTM: 10.12 Hybrid: 11.42 CL-WR: 7.97
477 -- FF: 15.44 LSTM: 10.15 Hybrid: 11.27 CL-WR: 8.0
478 -- FF: 15.92 LSTM: 9.92 Hybrid: 11.38 CL-WR: 8.0
479 -- FF: 17.64 LSTM: 10.24 Hybrid: 12.05 CL-WR: 8.04
480 -- FF: 16.65 LSTM: 10.58 Hybrid: 12.18 CL-WR: 7.2
481 -- FF: 15.98 LSTM: 10.51 Hybrid: 12.02 CL-WR: 6.88
482 -- FF: 16.9 LSTM: 11.41 Hybrid: 12.91 CL-WR: 6.46
483 -- FF: 16.28 LSTM: 11.0 Hybrid: 12.49 CL-WR: 6.44
484 -- FF: 15.99 LSTM: 10.84 Hybrid: 12.19 CL-WR: 6.54
485 -- FF: 16.34 LSTM: 10.79 Hybrid: 12.4 CL-WR: 6.41
486 -- FF: 16.38 LSTM: 11.0 Hybrid: 12.53 CL-WR: 6.62
487 -- FF: 16.11 LSTM: 10.67 Hybrid: 12.14 CL-WR: 6.91
488 -- FF: 15.39 LSTM: 10.27 Hybrid: 11.52 CL-WR: 7.42
489 -- FF: 15.8 LSTM: 10.56 Hybrid: 11.92 CL-WR: 7.26
490 -- FF: 17.69 LSTM: 10.51 Hybrid: 12.58 CL-WR: 7.48
491 -- FF: 15.79 LSTM: 10.7 Hybrid: 12.08 CL-WR: 7.25
492 -- FF: 15.87 LSTM: 10.69 Hybrid: 12.05 CL-WR: 7.27
493 -- FF: 16.18 LSTM: 10.79 Hybrid: 12.43 CL-WR: 7.1
494 -- FF: 15.51 LSTM: 10.42 Hybrid: 11.69 CL-WR: 7.78
495 -- FF: 16.46 LSTM: 10.69 Hybrid: 12.42 CL-WR: 7.28
496 -- FF: 16.16 LSTM: 10.47 Hybrid: 12.08 CL-WR: 7.57
497 -- FF: 17.09 LSTM: 10.66 Hybrid: 12.38 CL-WR: 7.84
498 -- FF: 16.75 LSTM: 10.46 Hybrid: 12.2 CL-WR: 7.57
499 -- FF: 15.79 LSTM: 10.5 Hybrid: 11.96 CL-WR: 7.15
500 -- FF: 15.48 LSTM: 10.71 Hybrid: 11.88 CL-WR: 7.83
501 -- FF: 16.33 LSTM: 11.05 Hybrid: 12.45 CL-WR: 7.41
502 -- FF: 15.28 LSTM: 10.56 Hybrid: 11.74 CL-WR: 7.75
503 -- FF: 16.02 LSTM: 10.45 Hybrid: 11.88 CL-WR: 7.77
504 -- FF: 15.33 LSTM: 10.4 Hybrid: 11.33 CL-WR: 8.55
505 -- FF: 16.26 LSTM: 10.79 Hybrid: 12.11 CL-WR: 7.62
506 -- FF: 16.41 LSTM: 11.13 Hybrid: 12.46 CL-WR: 6.98
507 -- FF: 17.37 LSTM: 11.71 Hybrid: 13.38 CL-WR: 6.36
508 -- FF: 16.03 LSTM: 11.33 Hybrid: 12.49 CL-WR: 6.81
509 -- FF: 16.41 LSTM: 11.34 Hybrid: 12.75 CL-WR: 6.64
510 -- FF: 16.25 LSTM: 11.4 Hybrid: 12.61 CL-WR: 6.67
511 -- FF: 17.04 LSTM: 11.51 Hybrid: 12.99 CL-WR: 6.73
512 -- FF: 17.59 LSTM: 11.52 Hybrid: 13.21 CL-WR: 6.98
513 -- FF: 15.37 LSTM: 10.56 Hybrid: 11.48 CL-WR: 8.13
514 -- FF: 15.92 LSTM: 10.38 Hybrid: 11.42 CL-WR: 8.5
515 -- FF: 15.86 LSTM: 10.23 Hybrid: 11.14 CL-WR: 9.09
516 -- FF: 17.62 LSTM: 11.24 Hybrid: 12.79 CL-WR: 8.34
517 -- FF: 16.16 LSTM: 10.29 Hybrid: 11.38 CL-WR: 8.78
518 -- FF: 15.64 LSTM: 10.22 Hybrid: 10.96 CL-WR: 9.27
519 -- FF: 15.48 LSTM: 10.15 Hybrid: 10.95 CL-WR: 9.04
520 -- FF: 15.97 LSTM: 10.37 Hybrid: 11.48 CL-WR: 8.87
521 -- FF: 16.38 LSTM: 10.35 Hybrid: 11.59 CL-WR: 8.81
522 -- FF: 16.4 LSTM: 10.42 Hybrid: 11.72 CL-WR: 8.63
523 -- FF: 16.11 LSTM: 10.3 Hybrid: 11.59 CL-WR: 8.91
524 -- FF: 16.54 LSTM: 10.18 Hybrid: 11.49 CL-WR: 9.29
525 -- FF: 16.4 LSTM: 10.41 Hybrid: 11.63 CL-WR: 9.15
526 -- FF: 16.62 LSTM: 10.4 Hybrid: 11.81 CL-WR: 8.77
527 -- FF: 16.86 LSTM: 11.07 Hybrid: 12.51 CL-WR: 7.95
528 -- FF: 16.75 LSTM: 10.83 Hybrid: 12.2 CL-WR: 8.14
529 -- FF: 16.43 LSTM: 10.71 Hybrid: 11.87 CL-WR: 8.52
530 -- FF: 16.38 LSTM: 10.32 Hybrid: 11.36 CL-WR: 8.68
531 -- FF: 17.29 LSTM: 11.1 Hybrid: 12.56 CL-WR: 8.56
532 -- FF: 16.34 LSTM: 10.58 Hybrid: 11.65 CL-WR: 8.64
533 -- FF: 16.47 LSTM: 10.62 Hybrid: 11.94 CL-WR: 8.1
534 -- FF: 16.25 LSTM: 10.78 Hybrid: 11.95 CL-WR: 8.27
535 -- FF: 16.78 LSTM: 11.24 Hybrid: 12.6 CL-WR: 8.23
536 -- FF: 16.89 LSTM: 10.92 Hybrid: 12.37 CL-WR: 8.23
537 -- FF: 16.08 LSTM: 11.1 Hybrid: 12.07 CL-WR: 8.46
538 -- FF: 17.06 LSTM: 10.87 Hybrid: 12.47 CL-WR: 8.36
539 -- FF: 17.68 LSTM: 11.69 Hybrid: 13.3 CL-WR: 8.5
540 -- FF: 16.76 LSTM: 11.09 Hybrid: 12.35 CL-WR: 8.61
541 -- FF: 16.32 LSTM: 10.86 Hybrid: 11.67 CL-WR: 9.1
542 -- FF: 16.08 LSTM: 10.76 Hybrid: 11.65 CL-WR: 9.09
543 -- FF: 15.54 LSTM: 10.85 Hybrid: 11.21 CL-WR: 9.17
544 -- FF: 15.58 LSTM: 10.67 Hybrid: 11.33 CL-WR: 9.45
545 -- FF: 15.95 LSTM: 10.89 Hybrid: 11.8 CL-WR: 9.13
546 -- FF: 11.01 LSTM: 10.31 Hybrid: 9.67 CL-WR: 10.32
547 -- FF: 11.22 LSTM: 10.83 Hybrid: 10.11 CL-WR: 10.64
548 -- FF: 11.28 LSTM: 10.58 Hybrid: 10.02 CL-WR: 10.34
549 -- FF: 11.06 LSTM: 10.6 Hybrid: 9.88 CL-WR: 10.74
550 -- FF: 11.15 LSTM: 10.51 Hybrid: 9.83 CL-WR: 10.16
551 -- FF: 11.15 LSTM: 10.98 Hybrid: 10.14 CL-WR: 10.67
552 -- FF: 11.09 LSTM: 10.61 Hybrid: 9.86 CL-WR: 10.79
553 -- FF: 11.22 LSTM: 10.87 Hybrid: 10.11 CL-WR: 10.7
554 -- FF: 11.34 LSTM: 10.27 Hybrid: 9.89 CL-WR: 9.91
555 -- FF: 11.18 LSTM: 10.45 Hybrid: 9.83 CL-WR: 9.57
556 -- FF: 11.07 LSTM: 11.05 Hybrid: 10.1 CL-WR: 10.44
557 -- FF: 10.95 LSTM: 10.73 Hybrid: 9.84 CL-WR: 10.16
558 -- FF: 10.96 LSTM: 10.89 Hybrid: 9.97 CL-WR: 10.03
559 -- FF: 10.81 LSTM: 10.76 Hybrid: 9.84 CL-WR: 10.4
560 -- FF: 11.42 LSTM: 10.35 Hybrid: 9.92 CL-WR: 9.01
561 -- FF: 11.63 LSTM: 10.53 Hybrid: 10.18 CL-WR: 8.7
562 -- FF: 11.6 LSTM: 10.32 Hybrid: 10.08 CL-WR: 8.49
563 -- FF: 11.43 LSTM: 10.13 Hybrid: 9.76 CL-WR: 8.81
564 -- FF: 11.36 LSTM: 10.06 Hybrid: 9.82 CL-WR: 8.79
565 -- FF: 11.5 LSTM: 10.28 Hybrid: 9.97 CL-WR: 8.64
566 -- FF: 11.38 LSTM: 10.06 Hybrid: 9.79 CL-WR: 8.54
567 -- FF: 11.01 LSTM: 10.32 Hybrid: 9.66 CL-WR: 9.28
568 -- FF: 10.85 LSTM: 10.65 Hybrid: 9.75 CL-WR: 9.83
569 -- FF: 10.74 LSTM: 10.43 Hybrid: 9.61 CL-WR: 10.25
570 -- FF: 10.82 LSTM: 10.8 Hybrid: 9.87 CL-WR: 10.13
571 -- FF: 10.6 LSTM: 10.65 Hybrid: 9.67 CL-WR: 10.53
572 -- FF: 10.66 LSTM: 10.48 Hybrid: 9.56 CL-WR: 9.99
573 -- FF: 10.8 LSTM: 10.36 Hybrid: 9.65 CL-WR: 9.64
574 -- FF: 10.78 LSTM: 10.92 Hybrid: 9.94 CL-WR: 10.42
575 -- FF: 10.78 LSTM: 10.28 Hybrid: 9.53 CL-WR: 9.62
576 -- FF: 10.78 LSTM: 10.39 Hybrid: 9.54 CL-WR: 9.44
577 -- FF: 10.79 LSTM: 10.45 Hybrid: 9.62 CL-WR: 9.31
578 -- FF: 10.74 LSTM: 10.6 Hybrid: 9.71 CL-WR: 9.82
579 -- FF: 10.96 LSTM: 10.36 Hybrid: 9.68 CL-WR: 9.34
580 -- FF: 11.32 LSTM: 9.85 Hybrid: 9.6 CL-WR: 8.17
581 -- FF: 11.06 LSTM: 9.76 Hybrid: 9.35 CL-WR: 8.56
582 -- FF: 11.17 LSTM: 10.31 Hybrid: 9.76 CL-WR: 8.48
583 -- FF: 10.58 LSTM: 10.49 Hybrid: 9.56 CL-WR: 10.04
584 -- FF: 10.57 LSTM: 10.59 Hybrid: 9.61 CL-WR: 9.95
585 -- FF: 10.42 LSTM: 10.81 Hybrid: 9.6 CL-WR: 10.81
586 -- FF: 10.69 LSTM: 10.22 Hybrid: 9.47 CL-WR: 9.33
587 -- FF: 11.47 LSTM: 9.96 Hybrid: 9.78 CL-WR: 7.75
588 -- FF: 11.84 LSTM: 10.02 Hybrid: 9.98 CL-WR: 7.46
589 -- FF: 11.9 LSTM: 9.93 Hybrid: 9.94 CL-WR: 7.29
590 -- FF: 11.84 LSTM: 9.49 Hybrid: 9.74 CL-WR: 7.02
591 -- FF: 11.71 LSTM: 9.64 Hybrid: 9.74 CL-WR: 7.04
592 -- FF: 11.56 LSTM: 9.8 Hybrid: 9.7 CL-WR: 7.49
593 -- FF: 11.33 LSTM: 9.79 Hybrid: 9.68 CL-WR: 7.97
594 -- FF: 11.05 LSTM: 10.03 Hybrid: 9.57 CL-WR: 8.52
595 -- FF: 10.59 LSTM: 10.77 Hybrid: 9.75 CL-WR: 9.98
596 -- FF: 10.54 LSTM: 10.08 Hybrid: 9.22 CL-WR: 9.83
597 -- FF: 15.92 LSTM: 10.34 Hybrid: 11.18 CL-WR: 8.99
598 -- FF: 10.53 LSTM: 9.84 Hybrid: 9.12 CL-WR: 8.92
599 -- FF: 10.26 LSTM: 10.67 Hybrid: 9.49 CL-WR: 10.07
600 -- FF: 10.5 LSTM: 10.41 Hybrid: 9.53 CL-WR: 9.81
601 -- FF: 10.53 LSTM: 11.07 Hybrid: 9.9 CL-WR: 10.53
602 -- FF: 10.23 LSTM: 10.6 Hybrid: 9.47 CL-WR: 10.86
603 -- FF: 10.5 LSTM: 10.35 Hybrid: 9.47 CL-WR: 10.21
604 -- FF: 10.24 LSTM: 10.52 Hybrid: 9.44 CL-WR: 10.49
605 -- FF: 10.36 LSTM: 9.7 Hybrid: 8.9 CL-WR: 9.86
606 -- FF: 10.33 LSTM: 10.4 Hybrid: 9.31 CL-WR: 10.35
607 -- FF: 10.63 LSTM: 9.91 Hybrid: 9.19 CL-WR: 9.51
608 -- FF: 10.6 LSTM: 9.75 Hybrid: 9.26 CL-WR: 9.29
609 -- FF: 10.33 LSTM: 9.97 Hybrid: 9.18 CL-WR: 9.95
610 -- FF: 10.2 LSTM: 10.1 Hybrid: 9.04 CL-WR: 10.32
611 -- FF: 10.15 LSTM: 10.05 Hybrid: 9.01 CL-WR: 10.44
612 -- FF: 11.33 LSTM: 10.14 Hybrid: 9.39 CL-WR: 10.04
613 -- FF: 10.1 LSTM: 10.25 Hybrid: 9.19 CL-WR: 10.51
614 -- FF: 10.81 LSTM: 9.59 Hybrid: 9.12 CL-WR: 8.58
615 -- FF: 10.85 LSTM: 9.62 Hybrid: 9.22 CL-WR: 8.37
616 -- FF: 10.92 LSTM: 9.66 Hybrid: 9.36 CL-WR: 8.71
617 -- FF: 10.95 LSTM: 9.38 Hybrid: 9.07 CL-WR: 8.11
618 -- FF: 10.92 LSTM: 9.48 Hybrid: 9.33 CL-WR: 8.34
619 -- FF: 11.01 LSTM: 9.61 Hybrid: 9.21 CL-WR: 8.26
620 -- FF: 10.78 LSTM: 9.52 Hybrid: 9.24 CL-WR: 8.63
621 -- FF: 12.42 LSTM: 9.61 Hybrid: 9.66 CL-WR: 8.45
622 -- FF: 10.59 LSTM: 9.3 Hybrid: 8.84 CL-WR: 8.52
623 -- FF: 12.35 LSTM: 9.06 Hybrid: 9.3 CL-WR: 8.33
624 -- FF: 10.7 LSTM: 9.04 Hybrid: 8.74 CL-WR: 8.02
625 -- FF: 10.9 LSTM: 9.37 Hybrid: 9.14 CL-WR: 8.08
626 -- FF: 10.88 LSTM: 9.14 Hybrid: 9.14 CL-WR: 7.83
627 -- FF: 11.36 LSTM: 9.43 Hybrid: 9.41 CL-WR: 7.63
628 -- FF: 10.8 LSTM: 9.2 Hybrid: 9.12 CL-WR: 8.23
629 -- FF: 11.24 LSTM: 9.31 Hybrid: 9.29 CL-WR: 7.53
630 -- FF: 10.93 LSTM: 9.2 Hybrid: 8.97 CL-WR: 8.07
631 -- FF: 10.64 LSTM: 9.43 Hybrid: 8.98 CL-WR: 8.66
632 -- FF: 12.44 LSTM: 9.05 Hybrid: 9.44 CL-WR: 7.9
633 -- FF: 10.52 LSTM: 9.27 Hybrid: 8.82 CL-WR: 8.74
634 -- FF: 10.14 LSTM: 9.22 Hybrid: 8.76 CL-WR: 8.87
635 -- FF: 9.95 LSTM: 9.27 Hybrid: 8.63 CL-WR: 9.31
636 -- FF: 10.79 LSTM: 9.11 Hybrid: 8.95 CL-WR: 8.15
637 -- FF: 10.76 LSTM: 10.46 Hybrid: 9.6 CL-WR: 11.0
638 -- FF: 9.86 LSTM: 9.81 Hybrid: 8.76 CL-WR: 10.01
639 -- FF: 9.6 LSTM: 10.17 Hybrid: 8.89 CL-WR: 10.81
640 -- FF: 9.76 LSTM: 9.88 Hybrid: 8.75 CL-WR: 10.42
641 -- FF: 9.54 LSTM: 9.55 Hybrid: 8.42 CL-WR: 9.73
642 -- FF: 10.07 LSTM: 9.1 Hybrid: 8.67 CL-WR: 8.77
643 -- FF: 9.9 LSTM: 8.91 Hybrid: 8.4 CL-WR: 9.53
644 -- FF: 10.76 LSTM: 9.17 Hybrid: 8.98 CL-WR: 8.17
645 -- FF: 9.94 LSTM: 9.62 Hybrid: 8.81 CL-WR: 10.18
646 -- FF: 10.94 LSTM: 9.58 Hybrid: 9.31 CL-WR: 9.44
647 -- FF: 10.3 LSTM: 8.93 Hybrid: 8.58 CL-WR: 8.45
648 -- FF: 10.82 LSTM: 9.08 Hybrid: 8.92 CL-WR: 8.43
649 -- FF: 13.47 LSTM: 10.24 Hybrid: 10.45 CL-WR: 10.36
650 -- FF: 9.83 LSTM: 9.53 Hybrid: 8.59 CL-WR: 9.9
651 -- FF: 9.26 LSTM: 10.0 Hybrid: 8.63 CL-WR: 12.7
652 -- FF: 9.83 LSTM: 10.55 Hybrid: 9.16 CL-WR: 12.83
653 -- FF: 9.89 LSTM: 10.62 Hybrid: 9.16 CL-WR: 12.91
654 -- FF: 13.8 LSTM: 9.73 Hybrid: 9.72 CL-WR: 10.5
655 -- FF: 9.73 LSTM: 10.86 Hybrid: 9.27 CL-WR: 12.97
656 -- FF: 9.11 LSTM: 10.35 Hybrid: 8.76 CL-WR: 12.35
657 -- FF: 9.74 LSTM: 10.48 Hybrid: 9.09 CL-WR: 12.22
658 -- FF: 9.18 LSTM: 10.45 Hybrid: 8.89 CL-WR: 13.4
659 -- FF: 9.65 LSTM: 10.99 Hybrid: 9.32 CL-WR: 13.4
660 -- FF: 9.41 LSTM: 9.8 Hybrid: 8.57 CL-WR: 11.75
661 -- FF: 10.24 LSTM: 9.18 Hybrid: 8.48 CL-WR: 9.1
662 -- FF: 9.95 LSTM: 10.41 Hybrid: 9.26 CL-WR: 11.76
663 -- FF: 8.92 LSTM: 9.7 Hybrid: 8.32 CL-WR: 12.04
664 -- FF: 9.27 LSTM: 9.29 Hybrid: 8.26 CL-WR: 10.55
665 -- FF: 8.86 LSTM: 9.27 Hybrid: 8.05 CL-WR: 11.52
666 -- FF: 9.98 LSTM: 10.22 Hybrid: 9.15 CL-WR: 11.27
667 -- FF: 10.13 LSTM: 10.36 Hybrid: 9.26 CL-WR: 11.81
668 -- FF: 10.6 LSTM: 9.44 Hybrid: 8.43 CL-WR: 11.62
669 -- FF: 9.78 LSTM: 10.45 Hybrid: 9.14 CL-WR: 12.03
670 -- FF: 9.11 LSTM: 9.25 Hybrid: 8.13 CL-WR: 9.51
671 -- FF: 9.34 LSTM: 8.88 Hybrid: 8.13 CL-WR: 9.21
672 -- FF: 10.02 LSTM: 8.98 Hybrid: 8.48 CL-WR: 8.36
673 -- FF: 9.33 LSTM: 8.89 Hybrid: 8.12 CL-WR: 8.75
674 -- FF: 9.05 LSTM: 8.88 Hybrid: 7.97 CL-WR: 9.66
675 -- FF: 9.18 LSTM: 8.82 Hybrid: 8.06 CL-WR: 9.52
676 -- FF: 10.08 LSTM: 9.42 Hybrid: 8.87 CL-WR: 9.63
677 -- FF: 9.68 LSTM: 9.58 Hybrid: 8.72 CL-WR: 10.06
678 -- FF: 9.56 LSTM: 10.05 Hybrid: 8.85 CL-WR: 11.28
679 -- FF: 8.59 LSTM: 10.0 Hybrid: 8.29 CL-WR: 12.45
680 -- FF: 9.59 LSTM: 10.54 Hybrid: 9.11 CL-WR: 11.96
681 -- FF: 9.3 LSTM: 10.47 Hybrid: 8.91 CL-WR: 12.46
682 -- FF: 9.54 LSTM: 10.68 Hybrid: 9.17 CL-WR: 12.22
683 -- FF: 9.55 LSTM: 10.55 Hybrid: 9.07 CL-WR: 11.64
684 -- FF: 8.37 LSTM: 10.29 Hybrid: 8.39 CL-WR: 11.71
685 -- FF: 9.59 LSTM: 10.5 Hybrid: 9.09 CL-WR: 11.31
686 -- FF: 9.59 LSTM: 10.78 Hybrid: 9.19 CL-WR: 11.31
687 -- FF: 8.64 LSTM: 9.31 Hybrid: 7.94 CL-WR: 10.08
688 -- FF: 8.37 LSTM: 10.06 Hybrid: 8.25 CL-WR: 11.29
689 -- FF: 9.63 LSTM: 10.32 Hybrid: 9.0 CL-WR: 11.08
690 -- FF: 8.2 LSTM: 9.84 Hybrid: 8.07 CL-WR: 11.27
691 -- FF: 8.32 LSTM: 9.67 Hybrid: 8.06 CL-WR: 10.89
692 -- FF: 9.95 LSTM: 10.08 Hybrid: 9.09 CL-WR: 9.95
693 -- FF: 9.95 LSTM: 9.8 Hybrid: 8.96 CL-WR: 10.06
694 -- FF: 9.17 LSTM: 9.0 Hybrid: 7.92 CL-WR: 8.33
695 -- FF: 8.37 LSTM: 9.1 Hybrid: 7.72 CL-WR: 8.74
696 -- FF: 9.44 LSTM: 10.4 Hybrid: 8.94 CL-WR: 11.26
697 -- FF: 7.9 LSTM: 9.55 Hybrid: 7.76 CL-WR: 11.3
698 -- FF: 9.78 LSTM: 10.52 Hybrid: 9.16 CL-WR: 11.6
699 -- FF: 9.75 LSTM: 10.18 Hybrid: 9.01 CL-WR: 11.8
700 -- FF: 9.95 LSTM: 9.47 Hybrid: 8.77 CL-WR: 10.27
701 -- FF: 9.29 LSTM: 9.22 Hybrid: 8.14 CL-WR: 9.14
702 -- FF: 8.61 LSTM: 9.0 Hybrid: 7.84 CL-WR: 9.4
703 -- FF: 8.37 LSTM: 9.16 Hybrid: 7.76 CL-WR: 10.25
704 -- FF: 9.76 LSTM: 9.41 Hybrid: 8.29 CL-WR: 9.05
705 -- FF: 8.88 LSTM: 9.03 Hybrid: 8.04 CL-WR: 8.32
706 -- FF: 8.01 LSTM: 9.5 Hybrid: 7.77 CL-WR: 10.22
707 -- FF: 8.47 LSTM: 9.35 Hybrid: 7.98 CL-WR: 9.46
708 -- FF: 8.14 LSTM: 9.72 Hybrid: 7.78 CL-WR: 11.28
709 -- FF: 7.66 LSTM: 9.43 Hybrid: 7.56 CL-WR: 10.55
710 -- FF: 8.24 LSTM: 9.4 Hybrid: 7.58 CL-WR: 10.98
711 -- FF: 8.48 LSTM: 9.18 Hybrid: 7.64 CL-WR: 10.39
712 -- FF: 7.75 LSTM: 9.01 Hybrid: 7.38 CL-WR: 9.94
713 -- FF: 8.57 LSTM: 9.49 Hybrid: 7.83 CL-WR: 10.54
714 -- FF: 7.6 LSTM: 9.51 Hybrid: 7.61 CL-WR: 11.0
715 -- FF: 7.48 LSTM: 8.89 Hybrid: 7.15 CL-WR: 10.46
716 -- FF: 9.85 LSTM: 9.44 Hybrid: 8.57 CL-WR: 9.72
717 -- FF: 7.66 LSTM: 9.2 Hybrid: 7.49 CL-WR: 10.34
718 -- FF: 7.4 LSTM: 9.06 Hybrid: 7.29 CL-WR: 10.59
719 -- FF: 7.45 LSTM: 9.41 Hybrid: 7.5 CL-WR: 10.95
720 -- FF: 8.51 LSTM: 9.17 Hybrid: 7.67 CL-WR: 10.29
721 -- FF: 8.54 LSTM: 9.48 Hybrid: 7.83 CL-WR: 10.44
722 -- FF: 8.93 LSTM: 9.05 Hybrid: 7.77 CL-WR: 9.62
723 -- FF: 8.8 LSTM: 9.61 Hybrid: 8.03 CL-WR: 10.72
724 -- FF: 7.27 LSTM: 9.25 Hybrid: 7.34 CL-WR: 10.94
725 -- FF: 7.36 LSTM: 9.6 Hybrid: 7.43 CL-WR: 10.89
726 -- FF: 7.6 LSTM: 10.26 Hybrid: 8.12 CL-WR: 12.95
727 -- FF: 7.15 LSTM: 9.32 Hybrid: 7.37 CL-WR: 11.9
728 -- FF: 9.73 LSTM: 9.73 Hybrid: 8.82 CL-WR: 11.91
729 -- FF: 7.3 LSTM: 9.11 Hybrid: 7.27 CL-WR: 11.73
730 -- FF: 8.13 LSTM: 9.57 Hybrid: 8.0 CL-WR: 10.31
731 -- FF: 7.97 LSTM: 9.79 Hybrid: 8.1 CL-WR: 10.6
732 -- FF: 6.86 LSTM: 8.8 Hybrid: 7.0 CL-WR: 9.91
733 -- FF: 7.31 LSTM: 8.88 Hybrid: 7.29 CL-WR: 9.75
734 -- FF: 7.21 LSTM: 9.25 Hybrid: 7.39 CL-WR: 10.2
735 -- FF: 6.6 LSTM: 9.46 Hybrid: 7.15 CL-WR: 12.73
736 -- FF: 6.43 LSTM: 9.33 Hybrid: 7.0 CL-WR: 11.3
737 -- FF: 6.39 LSTM: 8.92 Hybrid: 6.64 CL-WR: 11.68
738 -- FF: 6.88 LSTM: 9.13 Hybrid: 7.13 CL-WR: 11.95
739 -- FF: 6.02 LSTM: 9.05 Hybrid: 6.59 CL-WR: 12.3
740 -- FF: 6.24 LSTM: 9.07 Hybrid: 6.77 CL-WR: 11.92
741 -- FF: 6.41 LSTM: 9.62 Hybrid: 7.21 CL-WR: 12.74
742 -- FF: 6.35 LSTM: 9.03 Hybrid: 6.84 CL-WR: 11.95
743 -- FF: 6.43 LSTM: 9.26 Hybrid: 7.01 CL-WR: 11.66
744 -- FF: 8.2 LSTM: 10.49 Hybrid: 8.6 CL-WR: 11.17
"""

In [32]:
import re
import numpy as np


# Initialize lists to hold values
ff, lstm, hybrid, clwr = [], [], [], []

# Regex to extract values
pattern = r"FF:\s*([\d.]+)\s*LSTM:\s*([\d.]+)\s*Hybrid:\s*([\d.]+)\s*CL-WR:\s*([\d.]+)"

for match in re.finditer(pattern, log):
    f, l, h, c = map(float, match.groups())
    ff.append(f)
    lstm.append(l)
    hybrid.append(h)
    clwr.append(c)

# Compute averages
print(f"Average FF error:     {np.mean(ff):.2f}")
print(f"Average LSTM error:   {np.mean(lstm):.2f}")
print(f"Average Hybrid error: {np.mean(hybrid):.2f}")
print(f"Average CL-WR error:  {np.mean(clwr):.2f}")


Average FF error:     15.01
Average LSTM error:   11.44
Average Hybrid error: 12.19
Average CL-WR error:  7.76


In [15]:
import numpy as np

# Example data
X = np.array([0, 1, 2, 3, 4, 5])
y = np.array([1,2,3,4,5,6])
lookback = 3

# Show what sequences are being fed into LSTM and what label is expected
print("Training sequences (with targets):")
for i in range(len(X) - lookback):
    input_indices = list(range(i, i + lookback))             # indices used as input
    target_index = i + lookback                              # the "next" label
    print(f"Input indices: {input_indices}  →  Predict y[{target_index}] = {y[target_index]}")

# Show what LSTM outputs map to in evaluation
print("\nPrediction-time alignment check:")
lstm_predictions = []
labels_used = y[lookback:]

for j in range(len(y) - lookback):
    predicted_index = lookback + j
    print(f"Predicted y[{predicted_index}] = {y[predicted_index]} → label[lookback:][{j}] = {labels_used[j]}")


Training sequences (with targets):
Input indices: [0, 1, 2]  →  Predict y[3] = 4
Input indices: [1, 2, 3]  →  Predict y[4] = 5
Input indices: [2, 3, 4]  →  Predict y[5] = 6

Prediction-time alignment check:
Predicted y[3] = 4 → label[lookback:][0] = 4
Predicted y[4] = 5 → label[lookback:][1] = 5
Predicted y[5] = 6 → label[lookback:][2] = 6


In [22]:
print("Input: ", X[0:2])
print("Pred: ", X[1:3])
print("Extracted: ", X[1:3][-1])
print("Label: ", X[2:])


Input:  [0 1]
Pred:  [1 2]
Extracted:  2
Label:  [2 3 4 5]
